In [ ]:
import os
import shapely
import fiona
import geopandas as gpd
import pandas as pd
import matplotlib as plt
from pysld.style import StyleSld
from geo.Geoserver import Geoserver

In [ ]:
!rm -r /home/ajai-krishna/work/qgis_geoserver/Styles/Districts_style/*

In [ ]:
data_dir_dist = '/home/ajai-krishna/work/qgis_geoserver/India district Data'
save_dist_dir = os.path.join('/home/ajai-krishna/work/qgis_geoserver/Styles/Districts_style/')
style_dist_files = [os.path.join(save_dist_dir,i) for i in os.listdir(save_dist_dir)]
dist_files = [os.path.join(data_dir_dist,files) for files in os.listdir(data_dir_dist)]
dist_files

In [ ]:
data_dist_file = []
for files in dist_files:
    if os.path.basename(files).endswith('2021-2040.gpkg'):
        data_dist_file.append(files)

In [ ]:
len(data_dist_file)

In [ ]:
server = Geoserver('http://localhost:8080/geoserver', username='admin', password='geoserver')

for i in data_dist_file:
    pr_indices = ['jjas','ond','pr-mm','rainyday-frequency','simple-daily-indices']
    file_name = '_'.join(i.split('_')[-4:-1])
    print(file_name)

In [ ]:
stores={}
styles={}

for i in data_dist_file:
    pr_indices = ['jjas','ond','pr-mm','rainyday-frequency','simple-daily-indices']
    file_name = '_'.join(i.split('_')[-4:-1])
    
    datastore_name = os.path.splitext(os.path.basename(i))[0]
    try:
        server.delete_featurestore(featurestore_name = datastore_name, workspace = 'Climate_data_Indian_Districts')
        print(f'deleted {datastore_name}')
    except:
        print(f"good")
    store = server.create_gpkg_datastore(path = i)
    stores[datastore_name] = store
    print(f"created data_store:{datastore_name}")

    gdf = gpd.read_file(i)
    layers = fiona.listlayers(i)
    layer_name = layers[0] if layers else datastore_name
    
    # Decide palette
    if file_name.split("_")[1].lower().startswith(tuple(pr_indices)):
        palette = 'Greens'
    else:
        palette = 'Reds'

    mean_values = gdf['mean'].to_list()
    mean_values.append( max(mean_values)+0.01)
    mean_values.append(min(mean_values)-0.01)
    sld = StyleSld(
        style_name='polygonStyle',
        geom_type='polygon',
        attribute_name='mean',
        values=mean_values,
        number_of_class= 8,
        classification_method='equal_interval',
        color_palette=palette
    )

    # style=sld.generate_categorized_style()
    style =sld.generate_classified_style()
    

    sld_path = os.path.join(save_dist_dir, f"simple_style_{file_name}.sld")
    with open(sld_path, 'w') as f:
        f.write(style)
    print(f"Created SLD for {file_name} → {palette}")
    
    try:
        server.delete_style(style_name = file_name, workspace = 'Climate_data_Indian_Districts')
        print(f'deleted {file_name}')
    except Exception:
        print(f'style {file_name} not found, skipping delete')   
    
    style1 = server.upload_style(path = sld_path, name = file_name, workspace = 'Climate_data_Indian_Districts')
    styles[file_name] = style1
    print(f'style uploaded {file_name}')
    
    # layer_name = datastore_name
    server.publish_style(layer_name= layer_name, style_name = file_name ,workspace = 'Climate_data_Indian_Districts')
    print(f'published {layer_name} to {datastore_name}')
